# Выполнил: Филоненко Никита Дмитриевич

-----

QR-код (ссылка на репозиторий с проектом)


# Оглавление

[Описание тестового задания](#0.1)
1. [Сбор данных через arXiv API](#01)
2. [EDA](#02)
    - [Получение общего представления о данных](#02.1)
    - [Предобработка текста для анализа](#02.2)


## Описание тестового задания <a name="0.1"></a>

Задача - выделить основные тренды в видео генерации в 2024 году.

1. Получить метаданные статей с arXiv (любым способом) по теме “видео генерация”, выпущенных в 2024 году.
    
2. Написать код для выделения основных тем/трендов.
    
- Будет плюсом, если каждая тема будет представлена связным словосочетанием/предложением

3. Представить визуализацию результатов.
    


# Решение

### Импортируем необходимые модули 

In [151]:
import pandas as pd
import numpy as np
import arxiv
import matplotlib.pyplot as plt
import seaborn as sns

для получения метаданных статей из arXiv будем использовать его API, вместо написания парсера

поскольку:

- у arXiv нет жестких ограничений, платных тарифов

- API предоставляет доступ ко всему необходимому для выполнения данного задания

- лимит в 10 000 результатов на запрос – этого более чем достаточно

- соответственно нет жесткого rate limit, если не спамить слишком часто

- использование API избавит от лишней предобработки текста (например html теги)



## 1. Сбор данных через arXiv API <a name="01"></a>

будем использовать библиотеку arxiv, которая предоставляет удобный высокоуровневый интерфейс для работы с API arXiv

In [174]:
query = (
    'all:video generation OR all:video synthesis OR '
    'all:diffusion video OR all:GAN video OR '
    'all:text-to-video OR all:image-to-video OR '
    'all:scene generation '
    'AND submittedDate:[20240101 TO 20241231]'
)

search = arxiv.Search(
    query=query,
    max_results=150000,
    sort_by=arxiv.SortCriterion.Relevance,
    sort_order=arxiv.SortOrder.Ascending
)

выполним ленивую загрузку метаданных, создав генератор через `client.results(search)`, благодаря чему с экономим на оперативной памяти и времени начальной загрузки данных

также, учитывая тот факт, что arXiv API не позволяет фильтровать запросы по конкретному году (в нашем случае — 2024), при формировании набора данных дополнительно проверим соответствие года публикации, чтобы гарантировать включение только статей, опубликованных за 2024 год

In [175]:
client = arxiv.Client(
    page_size=500,
    delay_seconds=5,
    num_retries=5
)

papers = []

current_year = 2024

try:
    for result in client.results(search):
        if result.published.year == current_year:
            papers.append({
                "title": result.title,
                "abstract": result.summary,
                "published": result.published,
                "authors": [a.name for a in result.authors],
                "url": result.entry_id
            })
except arxiv.UnexpectedEmptyPageError:
    print("Достигнут предел доступных результатов")

df = pd.DataFrame(papers)

Достигнут предел доступных результатов


In [203]:
from colorama import Fore, Style

print(f"Сколько всего статей 2024 года получено: {Fore.GREEN}{len(df)}{Style.RESET_ALL}")

Сколько всего статей 2024 года получено: 2100


In [137]:
df

,title,abstract,published,authors,url
0,Video to Video Generative Adversarial Network ...,The development of sophisticated models for vi...,2024-10-28 01:35:10+00:00,"[Yintai Ma, Diego Klabjan, Jean Utke]",http://arxiv.org/abs/2410.20657v1
1,Contrastive Sequential-Diffusion Learning: Non...,Generated video scenes for action-centric sequ...,2024-07-16 15:03:05+00:00,"[Vasco Ramos, Yonatan Bitton, Michal Yarom, Id...",http://arxiv.org/abs/2407.11814v3
2,ReCapture: Generative Video Camera Controls fo...,"Recently, breakthroughs in video modeling have...",2024-11-07 18:59:45+00:00,"[David Junhao Zhang, Roni Paiss, Shiran Zada, ...",http://arxiv.org/abs/2411.05003v1
3,"StreamingT2V: Consistent, Dynamic, and Extenda...",Text-to-video diffusion models enable the gene...,2024-03-21 18:27:29+00:00,"[Roberto Henschel, Levon Khachatryan, Daniil H...",http://arxiv.org/abs/2403.14773v1
4,FreeLong: Training-Free Long Video Generation ...,Video diffusion models have made substantial p...,2024-07-29 11:52:07+00:00,"[Yu Lu, Yuanzhi Liang, Linchao Zhu, Yi Yang]",http://arxiv.org/abs/2407.19918v1
...,...,...,...,...,...
2095,Recording dynamic facial micro-expressions wit...,We present an approach of utilizing a multi-ca...,2024-10-02 19:30:21+00:00,"[Lucas Kreiss, Weiheng Tang, Ramana Balla, Xi ...",http://arxiv.org/abs/2410.01973v1
2096,FLAASH: Flow-Attention Adaptive Semantic Hiera...,The proliferation of tobacco-related content o...,2024-10-25 17:20:22+00:00,"[Naga VS Raviteja Chappa, Page Daniel Dobbs, B...",http://arxiv.org/abs/2410.19896v2
2097,MSEG-VCUQ: Multimodal SEGmentation with Enhanc...,High-speed video (HSV) phase detection (PD) se...,2024-11-12 00:54:26+00:00,"[Chika Maduabuchi, Ericmoore Jossou, Matteo Bu...",http://arxiv.org/abs/2411.07463v4
2098,Principles of Visual Tokens for Efficient Vide...,Video understanding has made huge strides in r...,2024-11-20 14:09:47+00:00,"[Xinyue Hao, Gen Li, Shreyank N Gowda, Robert ...",http://arxiv.org/abs/2411.13626v2


In [148]:
df.to_csv("data/data_arxiv_video_generation_2024.csv", index=False)

## 2. EDA <a name="02"></a>

#### 2.1 общее представление полученных данных

In [190]:
df = pd.read_csv(
    'data/data_arxiv_video_generation_2024.csv', 
    sep=',',
    engine='python',
    encoding='utf'
)

df.shape

(2100, 5)

In [191]:
df.head()

,title,abstract,published,authors,url
0,Video to Video Generative Adversarial Network ...,The development of sophisticated models for vi...,2024-10-28 01:35:10+00:00,"['Yintai Ma', 'Diego Klabjan', 'Jean Utke']",http://arxiv.org/abs/2410.20657v1
1,Contrastive Sequential-Diffusion Learning: Non...,Generated video scenes for action-centric sequ...,2024-07-16 15:03:05+00:00,"['Vasco Ramos', 'Yonatan Bitton', 'Michal Yaro...",http://arxiv.org/abs/2407.11814v3
2,ReCapture: Generative Video Camera Controls fo...,"Recently, breakthroughs in video modeling have...",2024-11-07 18:59:45+00:00,"['David Junhao Zhang', 'Roni Paiss', 'Shiran Z...",http://arxiv.org/abs/2411.05003v1
3,"StreamingT2V: Consistent, Dynamic, and Extenda...",Text-to-video diffusion models enable the gene...,2024-03-21 18:27:29+00:00,"['Roberto Henschel', 'Levon Khachatryan', 'Dan...",http://arxiv.org/abs/2403.14773v1
4,FreeLong: Training-Free Long Video Generation ...,Video diffusion models have made substantial p...,2024-07-29 11:52:07+00:00,"['Yu Lu', 'Yuanzhi Liang', 'Linchao Zhu', 'Yi ...",http://arxiv.org/abs/2407.19918v1


In [192]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      2100 non-null   object
 1   abstract   2100 non-null   object
 2   published  2100 non-null   object
 3   authors    2100 non-null   object
 4   url        2100 non-null   object
dtypes: object(5)
memory usage: 82.2+ KB


In [193]:
df.dtypes

title        object
abstract     object
published    object
authors      object
url          object
dtype: object

In [194]:
df.duplicated(keep='first').sum()

np.int64(0)

In [195]:
df.describe()

,title,abstract,published,authors,url
count,2100,2100,2100,2100,2100
unique,2100,2100,2098,2091,2100
top,Self-supervised Video Instance Segmentation Ca...,Tracking geographic entities from historical m...,2024-03-26 17:59:52+00:00,"['Dengsheng Chen', 'Jie Hu', 'Xiaoming Wei', '...",http://arxiv.org/abs/2411.17425v1
freq,1,1,2,2,1


In [196]:
df.isna().sum()

title        0
abstract     0
published    0
authors      0
url          0
dtype: int64

In [197]:
df['title'][0]

'Video to Video Generative Adversarial Network for Few-shot Learning Based on Policy Gradient'

In [198]:
df['abstract'][0]

'The development of sophisticated models for video-to-video synthesis has been\nfacilitated by recent advances in deep reinforcement learning and generative\nadversarial networks (GANs). In this paper, we propose RL-V2V-GAN, a new deep\nneural network approach based on reinforcement learning for unsupervised\nconditional video-to-video synthesis. While preserving the unique style of the\nsource video domain, our approach aims to learn a mapping from a source video\ndomain to a target video domain. We train the model using policy gradient and\nemploy ConvLSTM layers to capture the spatial and temporal information by\ndesigning a fine-grained GAN architecture and incorporating spatio-temporal\nadversarial goals. The adversarial losses aid in content translation while\npreserving style. Unlike traditional video-to-video synthesis methods requiring\npaired inputs, our proposed approach is more general because it does not\nrequire paired inputs. Thus, when dealing with limited videos in the

In [199]:
df['published'] = pd.to_datetime(df['published'])

df['year_publisher'] = df['published'].dt.year
df['month_publisher'] = df['published'].dt.month

df.drop(columns=['published'], axis=1, inplace=True)

In [200]:
df[['year_publisher', 'month_publisher']]

,year_publisher,month_publisher
0,2024,10
1,2024,7
2,2024,11
3,2024,3
4,2024,7
...,...,...
2095,2024,10
2096,2024,10
2097,2024,11
2098,2024,11


In [201]:
df['year_publisher'].unique()

array([2024], dtype=int32)

In [202]:
df['month_publisher'].value_counts()

month_publisher
12    309
11    216
10    209
3     204
6     194
7     167
5     163
9     159
4     145
8     134
1     102
2      98
Name: count, dtype: int64